# JDE ETL Source Design
## Goal:  Generate source SQL with friendly names and built-in data Conversion
1. Pull *ALL* Field metadata based on QA 9.3:  Name, Datatype, Decimals
2. Pull *Specific* Table fields
3. Create SQL mapiing pull with data-conversion

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import os, sys
import warnings

warnings.filterwarnings('ignore')

In [2]:
from decouple import config
config.search_path = '/home/jovyan/work'

### Connect to SQL DB

In [3]:
engine = create_engine(config('DATABASE_URL'))

### 1. Pull *ALL* Field metadata based on QA 9.3:  Name, Datatype, Decimals

In [4]:
sql_field_meta_server = 'ESYS_PROD'
sql_field_meta_lib = 'ARCPCOM71'

In [5]:
sql_field_meta = '''
SELECT 
	RTRIM("FRDTAI")				AS data_item
	,"FRDTAT"					AS data_item_type
	,"FROWTP"					AS data_type
	,"FRDTAS"					AS data_item_size
	,ISNULL("FRCDEC", 0)		AS display_decimals
	,ISNULL("FRDSCR", 'zNA')	AS row_description 
    
FROM 

    OPENQUERY ({}, '

	SELECT
		t.FRDTAI
		,FRDTAT
		,FROWTP
		,FRDTAS
		,FRCDEC
		,FRDSCR
	FROM
		{}.F9210 t
		LEFT JOIN {}.F9202 d
		ON t.FRDTAI = d.FRDTAI AND
			d.FRLNGP = '' '' AND
			d.FRSYR = '' ''  
')

'''.format(sql_field_meta_server, sql_field_meta_lib, sql_field_meta_lib)

In [6]:
print(sql_field_meta)


SELECT 
	RTRIM("FRDTAI")				AS data_item
	,"FRDTAT"					AS data_item_type
	,"FROWTP"					AS data_type
	,"FRDTAS"					AS data_item_size
	,ISNULL("FRCDEC", 0)		AS display_decimals
	,ISNULL("FRDSCR", 'zNA')	AS row_description 
    
FROM 

    OPENQUERY (ESYS_PROD, '

	SELECT
		t.FRDTAI
		,FRDTAT
		,FROWTP
		,FRDTAS
		,FRCDEC
		,FRDSCR
	FROM
		ARCPCOM71.F9210 t
		LEFT JOIN ARCPCOM71.F9202 d
		ON t.FRDTAI = d.FRDTAI AND
			d.FRLNGP = '' '' AND
			d.FRSYR = '' ''  
')




In [7]:
df_field_meta = pd.read_sql_query(sql_field_meta, engine);

In [8]:
df_field_meta.iloc[:,[2,3,4]] = df_field_meta.iloc[:,[2,3,4]].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_field_meta.fillna(value=0,inplace=True)

### 2. Pull *Specific* Table fields

#  Set Table Name HERE

In [27]:
sql_table = 'F5527'
#sql_table = 'F060116'
#sql_table = 'RPTRULES'

sql_link_server = 'ESYS_PROD'
sql_lib = 'ARCPDTA71'
#sql_lib = 'ARCPDTA71'
#sql_lib = 'ARCPTEL'
#sql_lib = 'ESEND'
#sql_lib = 'HSIPDTA71'

stage_db_schema = 'Integration.'
convert_julian_date = True

# [ARC | HSI] [P | D] [DTA | CDC] [ 71 | 94]


In [28]:
sql_table_fields = '''

SELECT * from OPENQUERY ({}, '
	SELECT
		*
	FROM
		QSYS2.SYSCOLUMNS
	WHERE
        TABLE_SCHEMA = ''{}'' AND
		TABLE_NAME in( ''{}'')
    ORDER BY 
        ORDINAL_POSITION
')

''' .format(sql_link_server, sql_lib, sql_table)

In [29]:
#print (sql_table_fields)

In [30]:
df_table_fields = pd.read_sql_query(sql_table_fields, engine);

#### Join table fields with data dictionary meta-data

In [31]:
#df_table_fields

In [32]:
df_table_fields = df_table_fields[['ORDINAL_POSITION', 'COLUMN_NAME', 'COLUMN_TEXT', 'DATA_TYPE','LENGTH', 'NUMERIC_PRECISION']]

In [33]:
df_table_fields['data_item'] = df_table_fields.COLUMN_NAME.str[2:]

In [34]:
df_table_fields.head()

,ORDINAL_POSITION,COLUMN_NAME,COLUMN_TEXT,DATA_TYPE,LENGTH,NUMERIC_PRECISION,data_item
0,1,QPCO,Company. . . . . . . . . . . . . . . . .,CHAR,5,NaN,CO
1,2,QPAC08,Market Segment . . . . . . . . . . . . .,CHAR,3,NaN,AC08
2,3,QPITM,Item Number (Short). . . . . . . . . . .,NUMERIC,8,8.0,ITM
3,4,QPUOM4,Pricing UOM. . . . . . . . . . . . . . .,CHAR,2,NaN,UOM4
4,5,QPOVPR,Override Price . . . . . . . . . . . . .,DECIMAL,15,15.0,OVPR


In [35]:
df_table_fields.data_item.unique()

array(['CO', 'AC08', 'ITM', 'UOM4', 'OVPR', '$RSC', '$CTR', '$TO', '$AID',
       'USER', 'PID', 'UPMJ', 'TDAY', 'JOBN'], dtype=object)

In [36]:
dff = pd.merge(df_table_fields,df_field_meta,on='data_item', how='left')

#### Cleanup final field name
remove trailing dots, special characters, and converto to lower_case

In [37]:
dff['row_description_final'] = dff.row_description.str.rstrip('. ').str.replace(r'%','pct').str.replace(r'$','amt').str.replace(r'[^0-9|a-z|" "]','', case=False).str.replace('  ',' ').str.lower().str.replace(' ','_')

#### Override with Defaults

In [38]:
dff['row_description_final'][dff['data_item']=='LITM'] = 'item_number'
dff['row_description_final'][dff['data_item']=='AN8'] = 'billto'
dff['row_description_final'][dff['data_item']=='SHAN'] = 'shipto'
dff['row_description_final'][dff['data_item']=='DOCO'] = 'salesorder_number'

### 3. Create SQL mapping pull with data-conversion

In [39]:
dff.groupby(['DATA_TYPE', 'data_type','display_decimals'])['ORDINAL_POSITION'].count()

DATA_TYPE  data_type  display_decimals
CHAR       2.0        0.0                 9
DECIMAL    9.0        0.0                 2
                      2.0                 1
NUMERIC    9.0        0.0                 1
           11.0       0.0                 1
Name: ORDINAL_POSITION, dtype: int64

In [40]:
dff

,ORDINAL_POSITION,COLUMN_NAME,COLUMN_TEXT,DATA_TYPE,LENGTH,NUMERIC_PRECISION,data_item,data_item_type,data_type,data_item_size,display_decimals,row_description,row_description_final
0,1,QPCO,Company. . . . . . . . . . . . . . . . .,CHAR,5,NaN,CO,A,2.0,5.0,0.0,Company. . . . . . . . . . . . . . . . .,company
1,2,QPAC08,Market Segment . . . . . . . . . . . . .,CHAR,3,NaN,AC08,A,2.0,3.0,0.0,Market Segment . . . . . . . . . . . . .,market_segment
2,3,QPITM,Item Number (Short). . . . . . . . . . .,NUMERIC,8,8.0,ITM,S,9.0,8.0,0.0,Item Number (Short). . . . . . . . . . .,item_number_short
3,4,QPUOM4,Pricing UOM. . . . . . . . . . . . . . .,CHAR,2,NaN,UOM4,A,2.0,2.0,0.0,Pricing UOM. . . . . . . . . . . . . . .,pricing_uom
4,5,QPOVPR,Override Price . . . . . . . . . . . . .,DECIMAL,15,15.0,OVPR,P,9.0,15.0,2.0,Override Price . . . . . . . . . . . . .,override_price
5,6,QP$RSC,Reason Code . . . . . . . . .. . .. . .,CHAR,2,NaN,$RSC,A,2.0,2.0,0.0,Reason Code . . . . . . . . . . . .. . .,reason_code
6,7,QP$CTR,Competitor . . . . . . . . . . . . . . .,CHAR,10,NaN,$CTR,A,2.0,10.0,0.0,Competitor . . . . . . . . . . . . . . .,competitor
7,8,QP$TO,Total Number of Orders . . . . . . . . .,DECIMAL,15,15.0,$TO,P,9.0,15.0,0.0,Total Number of Orders . . . . . . . . .,total_number_of_orders
8,9,QP$AID,Authorized ID. . . . . . . . . . . . . .,CHAR,10,NaN,$AID,A,2.0,10.0,0.0,Authorized ID. . . . . . . . . . . . . .,authorized_id
9,10,QPUSER,User ID. . . . . . . . . . . . . . . . .,CHAR,10,NaN,USER,A,2.0,10.0,0.0,User ID. . . . . . . . . . . . . . . . .,user_id


In [41]:
def field_format_sel(col_name, col_type, col_dec):
    val =''

    if col_type == 9 :
        if col_dec > 0 :
            val = 'CAST(({})/{} AS DEC({},{})) AS {}'.format(col_name,10**col_dec,15,np.int(col_dec),col_name)
        else :
            val = col_name
    elif  col_type == 11 : 
        if convert_julian_date :
            val = 'CASE WHEN {} IS NOT NULL THEN DATE(DIGITS(DEC({}+ 1900000,7,0))) ELSE NULL END AS {}'.format(col_name, col_name, col_name)
        else :
            val = '{} as {}'.format(col_name,col_name)
    else : 
        val = col_name
    return val;

def field_format_map(col_name, col_descr, is_etl = False):
    val =''

    col_name_format = '{message:{fill}{align}{width}}'.format(message=col_name, fill='_', align='<', width=6)
    
    col_descr_format = col_descr
    if str.isdigit(col_descr_format[0]) :
        col_descr_format = "_" + col_descr_format
        
    if is_etl :
        val = '{}_{} AS {}'.format(col_name_format, col_descr_format, col_descr_format)
    else :
        val = '"{}" AS {}_{}'.format(col_name, col_name_format, col_descr_format)

    return val;



sql_field_map = ', '.join([field_format_map(x,y)  \
                           for x, y in zip(dff['COLUMN_NAME'], dff['row_description_final'])])

sql_field_etl = ', '.join([field_format_map(x,y,is_etl=True)  \
                           for x, y in zip(dff['COLUMN_NAME'], dff['row_description_final'])])
#
# ok
sql_field_sel = ', '.join([ field_format_sel(col_name, col_type, col_dec) \
                           for col_name, col_type, col_dec in zip(dff['COLUMN_NAME'], dff['data_type'], dff['display_decimals'])])


In [42]:
sql_field_sel

'QPCO, QPAC08, QPITM, QPUOM4, CAST((QPOVPR)/100.0 AS DEC(15,2)) AS QPOVPR, QP$RSC, QP$CTR, QP$TO, QP$AID, QPUSER, QPPID, CASE WHEN QPUPMJ IS NOT NULL THEN DATE(DIGITS(DEC(QPUPMJ+ 1900000,7,0))) ELSE NULL END AS QPUPMJ, QPTDAY, QPJOBN'

In [43]:
sql_table_map = '''

--------------------------------------------------------------------------------
-- DROP TABLE Integration.{}_{}_<instert_friendly_name_here>
--------------------------------------------------------------------------------

SELECT 

    {} 
    {} 

-- INTO {}{}_{}_<instert_friendly_name_here>

FROM 
    OPENQUERY ({}, '

	SELECT
		{}

	FROM
		{}.{}
--    WHERE
--        <insert custom code here>
--    ORDER BY
--        <insert custom code here>
')

--------------------------------------------------------------------------------

'''.format(sql_lib, sql_table, 'Top 5', sql_field_map, stage_db_schema, sql_lib, sql_table, sql_link_server, sql_field_sel, sql_lib, sql_table)




### Output Table
1. Use to create STAGE via SQL link table 
1. Note that some Julian _JDT conversion will need to be converted Post 

In [44]:
print(sql_table_map)



--------------------------------------------------------------------------------
-- DROP TABLE Integration.ARCPDTA71_F5527_<instert_friendly_name_here>
--------------------------------------------------------------------------------

SELECT 

    Top 5 
    "QPCO" AS QPCO___company, "QPAC08" AS QPAC08_market_segment, "QPITM" AS QPITM__item_number_short, "QPUOM4" AS QPUOM4_pricing_uom, "QPOVPR" AS QPOVPR_override_price, "QP$RSC" AS QP$RSC_reason_code, "QP$CTR" AS QP$CTR_competitor, "QP$TO" AS QP$TO__total_number_of_orders, "QP$AID" AS QP$AID_authorized_id, "QPUSER" AS QPUSER_user_id, "QPPID" AS QPPID__program_id, "QPUPMJ" AS QPUPMJ_date_updated, "QPTDAY" AS QPTDAY_time_of_day, "QPJOBN" AS QPJOBN_work_station_id 

-- INTO Integration.ARCPDTA71_F5527_<instert_friendly_name_here>

FROM 
    OPENQUERY (ESYS_PROD, '

	SELECT
		QPCO, QPAC08, QPITM, QPUOM4, CAST((QPOVPR)/100.0 AS DEC(15,2)) AS QPOVPR, QP$RSC, QP$CTR, QP$TO, QP$AID, QPUSER, QPPID, CASE WHEN QPUPMJ IS NOT NULL THEN DATE(DIGITS

In [45]:
dff

,ORDINAL_POSITION,COLUMN_NAME,COLUMN_TEXT,DATA_TYPE,LENGTH,NUMERIC_PRECISION,data_item,data_item_type,data_type,data_item_size,display_decimals,row_description,row_description_final
0,1,QPCO,Company. . . . . . . . . . . . . . . . .,CHAR,5,NaN,CO,A,2.0,5.0,0.0,Company. . . . . . . . . . . . . . . . .,company
1,2,QPAC08,Market Segment . . . . . . . . . . . . .,CHAR,3,NaN,AC08,A,2.0,3.0,0.0,Market Segment . . . . . . . . . . . . .,market_segment
2,3,QPITM,Item Number (Short). . . . . . . . . . .,NUMERIC,8,8.0,ITM,S,9.0,8.0,0.0,Item Number (Short). . . . . . . . . . .,item_number_short
3,4,QPUOM4,Pricing UOM. . . . . . . . . . . . . . .,CHAR,2,NaN,UOM4,A,2.0,2.0,0.0,Pricing UOM. . . . . . . . . . . . . . .,pricing_uom
4,5,QPOVPR,Override Price . . . . . . . . . . . . .,DECIMAL,15,15.0,OVPR,P,9.0,15.0,2.0,Override Price . . . . . . . . . . . . .,override_price
5,6,QP$RSC,Reason Code . . . . . . . . .. . .. . .,CHAR,2,NaN,$RSC,A,2.0,2.0,0.0,Reason Code . . . . . . . . . . . .. . .,reason_code
6,7,QP$CTR,Competitor . . . . . . . . . . . . . . .,CHAR,10,NaN,$CTR,A,2.0,10.0,0.0,Competitor . . . . . . . . . . . . . . .,competitor
7,8,QP$TO,Total Number of Orders . . . . . . . . .,DECIMAL,15,15.0,$TO,P,9.0,15.0,0.0,Total Number of Orders . . . . . . . . .,total_number_of_orders
8,9,QP$AID,Authorized ID. . . . . . . . . . . . . .,CHAR,10,NaN,$AID,A,2.0,10.0,0.0,Authorized ID. . . . . . . . . . . . . .,authorized_id
9,10,QPUSER,User ID. . . . . . . . . . . . . . . . .,CHAR,10,NaN,USER,A,2.0,10.0,0.0,User ID. . . . . . . . . . . . . . . . .,user_id


In [28]:
%time df_table_map = pd.read_sql_query(sql_table_map, engine);

CPU times: user 4.62 ms, sys: 84 µs, total: 4.7 ms
Wall time: 692 ms


### Next steps...
Add SQL to SQL Tools data package 

In [167]:
df_table_map

,QPCO___company,QPAC08_market_segment,QPITM__item_number_short,QPUOM4_pricing_uom,QPOVPR_override_price,QP$RSC_reason_code,QP$CTR_competitor,QP$TO__total_number_of_orders,QP$AID_authorized_id,QPUSER_user_id,QPPID__program_id,QPUPMJ_date_updated,QPTDAY_time_of_day,QPJOBN_work_station_id
0,02000,DEN,770473.0,EA,1.0,CP,,3.0,EKOLOD,EKOLOD,P551101,2009-07-21,153014.0,ONNLL3124
1,02000,DEN,768589.0,BX,1.0,CP,,1.0,BMAR04,BMAR04,P551101,2007-03-08,105724.0,QPADEV0062
2,02000,DEN,786726.0,KT,1.0,CP,,3.0,BMAR04,BMAR04,P551101,2007-03-08,110550.0,QPADEV0062
3,02000,DEN,771144.0,EA,1.0,CP,,4.0,BMAR04,BMAR04,P551101,2007-03-08,111755.0,QPADEV0062
4,02000,DEN,768589.0,BX,2.0,CP,,3.0,MMORIN,MMORIN,P551101,2007-03-14,100041.0,QPADEV007N
